In [1]:
# Cell 1: Install (run once)
!pip install datasets

In [2]:
# Load the SUMPUBMED dataset
from datasets import load_dataset
dataset = load_dataset("Blaise-g/SumPubmed")
print("Dataset loaded!")
print(dataset)

/Users/alexmusyoka/Projects/Gen and Collab AI/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset loaded!
DatasetDict({
    train: Dataset({
        features: ['line_text', 'filename_text', 'text', 'shorter_abstract', 'abstract'],
        num_rows: 26147
    })
    test: Dataset({
        features: ['line_text', 'filename_text', 'text', 'shorter_abstract', 'abstract'],
        num_rows: 3269
    })
    dev: Dataset({
        features: ['line_text', 'filename_text', 'text', 'shorter_abstract', 'abstract'],
        num_rows: 3268
    })
})


In [3]:
# Look at one example
example = dataset["train"][0]
print("PAPER TEXT (first 500 chars):")
print(example["text"][:500])
print("\n" + "="*50 + "\n")
print("ABSTRACT:")
print(example["abstract"])

PAPER TEXT (first 500 chars):
 thioredoxins are widely distributed in nature from prokaryotes to eukaryotes. these proteins, which belong to the oxidoreductase thiol:disulfide superfamily, are characterized by the active site signature sequence wcxxc. this sequence motif constitutes the redox center mediating the isomerization of specific disulfide bridges on trx target proteins. in yeasts and mammals, the cytoplasmic trx redox system is complemented by a second trx system within mitochondria. in plants, the system is more i


ABSTRACT:
 natrxh, a thioredoxin type h, shows differential expression between self-incompatible and self-compatible nicotiana species. natrxh interacts in vitro with s-rnase and co-localizes with it in the extracellular matrix of the stylar transmitting tissue. natrxh contains n- and c-terminal extensions, a feature shared by thioredoxin h proteins of subgroup to ascertain the function of these extensions in natrxh secretion and protein-protein interaction, we p

In [4]:
# Install PyTorch (required for running models)
!pip install torch

In [5]:
!pip install transformers==4.40.0

In [6]:
from transformers import BartForConditionalGeneration, BartTokenizer
# Load pre-trained BART (trained on CNN/DailyMail summarization)
model_name = "facebook/bart-large-cnn"
print("Loading tokenizer...")
tokenizer = BartTokenizer.from_pretrained(model_name)
print("Loading model... (this may take a minute)")
model = BartForConditionalGeneration.from_pretrained(model_name)
print("Model loaded!")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


Loading tokenizer...


/Users/alexmusyoka/Projects/Gen and Collab AI/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading model... (this may take a minute)
Model loaded!


In [7]:
# Task 3: Generate Baseline Summaries
# Test on a single example first
def generate_summary(text, max_length=150, min_length=50):
    """Generate a summary using pre-trained BART (no fine-tuning)."""
    inputs = tokenizer(
        text,
        return_tensors="pt",
        max_length=1024,
        truncation=True
    )
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        min_length=min_length,
        num_beams=4,
        early_stopping=True,
        no_repeat_ngram_size=3
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)
# Test on first test example
test_example = dataset["test"][0]
print("=" * 60)
print("INPUT (first 500 chars):")
print(test_example["text"][:500])
print("\n" + "=" * 60)
print("\nGROUND TRUTH ABSTRACT:")
print(test_example["abstract"][:500] + "...")
print("\n" + "=" * 60)
print("\nBASELINE GENERATED SUMMARY:")
baseline_summary = generate_summary(test_example["text"])
print(baseline_summary)

INPUT (first 500 chars):
 evolutionary molecular biology is mostly concerned with the forces affecting individual genes. however, observations of variable proportions of guanine and cytosine in different species and in different genomic regions of vertebrates have prompted the analysis of forces that may affect the evolution of complete genomes. one particular hypothesis concerns adaptation to high temperatures, proposing that high gc content results from selection favouring g:c pairs over less stable a:t pairs. against


GROUND TRUTH ABSTRACT:
 among bacteria and archaea, amino acid usage is correlated with habitat temperatures. in particular, protein surfaces in species thriving at higher temperatures appear to be enriched in amino acids that stabilize protein structure and depleted in amino acids that decrease thermostability. does this observation reflect a causal relationship, or could the apparent trend be caused by phylogenetic relatedness among sampled organisms living at diffe

In [ ]:
# Task 3: Generate Baseline Summaries (6-8 examples)
# Pick 8 samples spread across the test set
sample_indices = [0, 100, 500, 1000, 1500, 2000, 2500, 3000]
baseline_results = []
print("Generating baseline summaries...\n")
for i, idx in enumerate(sample_indices):
    example = dataset["test"][idx]
    generated = generate_summary(example["text"])
    
    baseline_results.append({
        "index": idx,
        "input_text": example["text"],
        "ground_truth": example["abstract"],
        "generated": generated
    })
    
    print(f"{'='*60}")
    print(f"SAMPLE {i+1} (index {idx})")
    print(f"{'='*60}")
    print(f"\nINPUT (first 300 chars):\n{example['text'][:300]}...")
    print(f"\nGROUND TRUTH (first 200 chars):\n{example['abstract'][:200]}...")
    print(f"\nGENERATED SUMMARY:\n{generated}")
    print()
print(f"\n✅ Generated {len(baseline_results)} baseline summaries")

Generating baseline summaries...

SAMPLE 1 (index 0)

INPUT (first 300 chars):
 evolutionary molecular biology is mostly concerned with the forces affecting individual genes. however, observations of variable proportions of guanine and cytosine in different species and in different genomic regions of vertebrates have prompted the analysis of forces that may affect the evolutio...

GROUND TRUTH (first 200 chars):
 among bacteria and archaea, amino acid usage is correlated with habitat temperatures. in particular, protein surfaces in species thriving at higher temperatures appear to be enriched in amino acids t...

GENERATED SUMMARY:
There seems to be no direct relationship between the gc content of prokaryotic protein-coding genes and optimal growth temperature. In vertebrates, it was argued convincingly that the'isochore' structure of high- and low-gc regions is not due to selection, but reflects varying fixation biases.

SAMPLE 2 (index 100)

INPUT (first 300 chars):
 the increasing a